This project is been developed as exam project in a context of "Business, Economic and Financial Data", at University of Padua (Italy).  
Developed by Carlo, Hau Ye and Nicolò, Master Students in Data Science.


# Introduction

In recent years, one of the most discussed topic has certainly been the pollution. There are many factors that characterize this problem, but among all, CO2 emissions are surely the stand out factor. The goal of this project is to predict these emissions among some of the most developed countries in the world.  
We will use related topics such as trade, gross domestic product (GDP) and urban population. After analyzing the topics and countries' datasets by extracting information and predictors, we will build some models that fit well with our problem and explain the results with both univariate and multivariate analisys.  
Moreover, for some models we will present some insights and explanations that could help in the understanding of trend's and model's behaviours


In [ ]:
library(dplyr)
library(tidyverse) # metapackage of all tidyverse packages
library(caret)
library(ggplot2)
library("patchwork")
library("ggforce")
library(corrplot)

library(forecast)
library(lmtest) 
library(fpp2)
library(zoo)

# Dataset

The datasets are in public domain available on World Bank (https://data.worldbank.org/). As mentioned we are going to use 4 datasets: 
* CO2 emissions, 
* Trade Openness, 
* GDP and 
* Urban Population  
All of them are filtered for the 10 most developed countries but we will take just USA, Italy and China. Some countries have missing values and we will treat this inadequacy differently.


In [ ]:
#data <- read.csv(file = "../input/top10-countries-co2gdptradeopeness/Complete DS 2.csv", dec=",")
# data <- read.csv(file = "../input/top10-countries-co2gdptradeopeness/Complete DS 2.csv", dec=",")
data <- read.csv(file = "../input/top10-countries-co2gdptradeopeness/Complete DS 2.csv", dec=",")
colnames(data)[2] <- "country_code"
data$country_code <- as.factor(data$country)
colnames(data)[3] <- "indicator"
data$indicator <- as.factor(data$indicator)
data$Country.Name <- NULL
data$Indicator.Code <- NULL
levels(data$indicator)

data[,3:ncol(data)] <- sapply(data[,3:ncol(data)], as.numeric)

head(data,10)

# Exploratory Data Analysis


## CO2 Emission in kt

Basically, the co2 emissions are measured with two metrics: 
* Consumption-based emissions take into account emissions embedded in international trade flows, and include emissions embedded in imports but exclude emissions from goods and services produced to serve overseas markets.
* Production-based emissions allocate emissions to production and include all emissions from domestic production, regardless of whether it is to serve domestic or overseas markets.  

Our dataset is based on ‘production’ or ‘territorial’ emissions and doesn't consider the emissions traded goods.


In [ ]:
CO2 <- data %>% filter (data$indicator == "CO2 emissions (kt)")
CO2$indicator <- NULL
Years <- data.frame(c(1960:2020))
coun = CO2$country_code

CO2 <- as.data.frame(t(as.matrix(CO2)))
colnames(CO2) <- coun
CO2 <- CO2[-1,]
rownames(CO2) <- NULL
CO2 <- cbind(CO2, Years)
colnames(CO2)[ncol(CO2)] <- "Years"
CO2 <- as.data.frame(sapply(CO2, as.numeric))


head(CO2)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)

ggplot(CO2, aes(x=Years))+
        geom_line(aes(y=CHN, color="CHN"),na.rm = TRUE)+
        geom_line(aes(y=USA, color="USA"),na.rm = TRUE)+        
        geom_line(aes(y=ITA, color="ITA"),na.rm = TRUE)+
        scale_color_discrete(name = "Countries")+
        ggtitle("CO2 emission in Kg")+
        ylab("CO2")+
        scale_y_continuous(labels = function(x) format(x, scientific = FALSE))

## GDP in USD$

Economic growth describes an increase in the quantity and quality of economic goods and services that a society produces and consumes. This growth is often measured as an increase in household income or inflation-adjusted GDP, but it is only an approximation. Income measures are just one way to understand economic inequality between countries and the change in prosperity over time. The GDP is the monetary value of all final goods and services produced within a country or region over a given period of time.


In [ ]:
GDP <- data %>% filter (data$indicator == "GDP (current US$)")
GDP$indicator <- NULL
Years <- data.frame(c(1960:2020))
coun = GDP$country_code

GDP <- as.data.frame(t(as.matrix(GDP)))
colnames(GDP) <- coun
GDP <- GDP[-1,]
rownames(GDP) <- NULL
GDP <- cbind(GDP, Years)
colnames(GDP)[ncol(GDP)] <- "Years"
GDP <- as.data.frame(sapply(GDP, as.numeric))


head(GDP)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)

ggplot(GDP, aes(x=Years))+
        geom_line(aes(y=CHN, color="CHN"),na.rm = TRUE)+
        geom_line(aes(y=USA, color="USA"),na.rm = TRUE)+        
        geom_line(aes(y=ITA, color="ITA"),na.rm = TRUE)+
        scale_color_discrete(name = "Countries")+
        ylab("GDP")+
        ggtitle("GDP in USD$")

## Trade Openness ratio (%of GDP)

The so-called trade openness index is an economic metric calculated as the ratio of the country's total trade to the country's GDP. This metric captures all transactions in income and outcome. The higher the index, the greater the influence of trade on domestic economic activities. This feature is also considered a significant variable in the evolution of environmental pollution. Many studies correlate trade openness with environmental pollution, the economic field (growth or crisis) and energy consumption. Furthermore, the effects of trade opening are linked to production, non-renewable energy use and polluting emissions and are considered an effective determinant of carbon emissions.


In [ ]:
TO <- data %>% filter (data$indicator == "Trade (% of GDP)")
TO$indicator <- NULL
Years <- data.frame(c(1960:2020))
coun = TO$country_code

TO <- as.data.frame(t(as.matrix(TO)))
colnames(TO) <- coun
TO <- TO[-1,]
rownames(TO) <- NULL
TO <- cbind(TO, Years)
colnames(TO)[ncol(TO)] <- "Years"
TO <- as.data.frame(sapply(TO, as.numeric))


head(TO)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)

ggplot(TO, aes(x=Years))+
        geom_line(aes(y=CHN, color="CHN"),na.rm = TRUE)+
        geom_line(aes(y=USA, color="USA"),na.rm = TRUE)+        
        geom_line(aes(y=ITA, color="ITA"),na.rm = TRUE)+
        scale_color_discrete(name = "Countries")+
        ylab("Trade Openess")+
        ggtitle("Trade Openess ratio (%of GDP)")

## Urban population

Urban population or urbanization refers to the development of existing cities and the emergence of new urban centers. It’s a process that has been advancing in the last few centuries. The most significant stage was the Industrial Revolution, when there were changes in the habits of today's civilization that contributed to global warming and consequent air pollution. In recent years, however, public investments in clean energy have increased and this could affect a slight decline in CO2 emissions.

In [ ]:
UP <- data %>% filter (data$indicator == "Urban population")
UP$indicator <- NULL
Years <- data.frame(c(1960:2020))
coun = UP$country_code

UP <- as.data.frame(t(as.matrix(UP)))
colnames(UP) <- coun
UP <- UP[-1,]
rownames(UP) <- NULL
UP <- cbind(UP, Years)
colnames(UP)[ncol(UP)] <- "Years"
UP <- as.data.frame(sapply(UP, as.numeric))


head(UP)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)

ggplot(UP, aes(x=Years))+
        geom_line(aes(y=CHN, color="CHN"),na.rm = TRUE)+
        geom_line(aes(y=USA, color="USA"),na.rm = TRUE)+        
        geom_line(aes(y=ITA, color="ITA"),na.rm = TRUE)+
        scale_color_discrete(name = "Countries")+
        ylab("Urban Population")+
        ggtitle("Urban Population")

## Merging the three dataset's rows grouped by country
Note that, as we will see, some time series will have missing values. We are going to analyze and model the data in both the complete version (from 1960 to 2020) and a truncated version (without NAs).  
In the modelling part we are going to use the former for the univariate analysis and the latter for the multvariate analysis, that is because NAs will be also present in the models' fitting.

In the plot below we are also going to normalize the features to allow them to stay all in the same plot

## USA dataset

In [ ]:
co2 <- CO2$USA
gdp <- GDP$USA
to <- TO$USA
up <- UP$USA
years <- CO2$Years
USA <- cbind.data.frame(co2, gdp, to, up, years)
USA_ts = ts(USA[-5],start = min(USA$years),end=max(USA$years))

In [ ]:
cat("NAs in CO2:",sum(is.na(USA$co2)), 
     "\nNAs in GDP:",sum(is.na(USA$gdp)),
     "\nNAs in TO:", sum(is.na(USA$to)),
     "\nNAs in UP:", sum(is.na(USA$up)),
     "\nNAs in years:", sum(is.na(USA$years)))
     

USA = USA[complete.cases(USA),]
row.names(USA) <- NULL #reset the index
head(USA)

In [ ]:
options(repr.plot.width=8, repr.plot.height=7)
par(mfrow=c(1,3))
plot(USA_ts)

The time series of the USA has smooth trends in the GDP and urban population features, but in the trend of trade openness is fluctuating and has initial NA values, in the co2 emissions feature the values begin to be high in the early '70.

In [ ]:
acf(USA[-5], na.action=na.pass)

As we can see from the autocorrelograms, the values of the time series are strongly correlated in the first lags and the influence decreases until it loses significance

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)
norm.USA <- cbind.data.frame(scale(USA[-5]), USA$years)

ggplot(norm.USA, aes(x=USA$years))+
        geom_line(aes(y=co2, color="CO2"),na.rm = TRUE) +
        geom_line(aes(y=gdp, color="GDP"),na.rm = TRUE)+
        geom_line(aes(y=to, color="Trade Op"),na.rm = TRUE)+
        geom_line(aes(y=up, color="Urban pop"),na.rm = TRUE)+
        scale_color_discrete(name = "Features", labels = c(
            "#CD5C5C"="CO2", "#4B0082"="GDP", "#000080"="Trade Op", "#7FFF00"="Urban pop"))+
        ggtitle("Normalized Features")

In [ ]:
options(repr.plot.width=6, repr.plot.height=7)
corr <- cor(USA[-5], use="pairwise.complete.obs")

col <- colorRampPalette(c("#BB4444", "#EE9988", "#FFFFFF", "#77AADD", "#4477AA"))
corrplot(corr, method="color", col=col(200), 
         type="lower", order="hclust", 
         addCoef.col = "black", 
         tl.col="black", tl.srt=40, 
         sig.level = 0.01, insig = "blank", 
         diag=FALSE)

The correlation plot shows a very high multicollinearity between the features.
Collinearity refers to the situation in which two or more predictor variables are closely related to one another.
Effects of collinearity:
- reduces the accuracy of estimates of the regression coefficients
- the standard erro for $ \beta_j $ grows
- the t-statistic declines -> we may fail to reject $ H0 : \beta_j = 0 $

it is possible for collinearity to exist between three or more variables, in this case it is called *multicollinearity*


## Italy dataset

In [ ]:
co2 <- CO2$ITA
gdp <- GDP$ITA
to <- TO$ITA
up <- UP$ITA
years <- CO2$Years
Italy <- cbind.data.frame(co2, gdp, to, up, years)
Italy_ts = ts(Italy[-5],start = min(Italy$years),end=max(Italy$years))
head(Italy)

In [ ]:
cat("NAs in CO2:",sum(is.na(Italy$co2)), 
     "\nNAs in GDP:",sum(is.na(Italy$gdp)),
     "\nNAs in TO:", sum(is.na(Italy$to)),
     "\nNAs in UP:", sum(is.na(Italy$up)),
     "\nNAs in years:", sum(is.na(Italy$years)))
     

Italy = Italy[complete.cases(Italy),]
row.names(Italy) <- NULL
head(Italy)

In [ ]:
options(repr.plot.width=8, repr.plot.height=7)
par(mfrow=c(1,3))
plot(Italy_ts)

In the time series of Italy the urban population has a smooth trend, but in the other features the trend changes frequently over time.

In [ ]:
acf(Italy[-5], na.action=na.pass)

We can observe almost the same behaviour as the USA's autocorrelograms

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)
norm.Ita <- cbind.data.frame(scale(Italy[-5]), Italy$years)

ggplot(norm.Ita, aes(x=Italy$years))+
        geom_line(aes(y=co2, color="CO2"),na.rm = TRUE) +
        geom_line(aes(y=gdp, color="GDP"),na.rm = TRUE)+
        geom_line(aes(y=to, color="Trade Op"),na.rm = TRUE)+
        geom_line(aes(y=up, color="Urban pop"),na.rm = TRUE)+
        scale_color_discrete(name = "Features", labels = c(
            "#CD5C5C"="CO2", "#4B0082"="GDP", "#000080"="Trade Op", "#7FFF00"="Urban pop"))+
        ylab("Features") +
        ggtitle("Normalized Features")

In [ ]:
options(repr.plot.width=6, repr.plot.height=7)
corr <- cor(Italy[-5], use="pairwise.complete.obs")

col <- colorRampPalette(c("#BB4444", "#EE9988", "#FFFFFF", "#77AADD", "#4477AA"))
corrplot(corr, method="color", col=col(200), 
         type="lower", order="hclust", 
         addCoef.col = "black", 
         tl.col="black", tl.srt=40, 
         sig.level = 0.01, insig = "blank", 
         diag=FALSE)

The correlation plot shows us that there isn't high correlations on the response variable, by the way we can notice that the explanatory variables are higly correlated between them.

### China dataset

In [ ]:
co2 <- CO2$CHN
gdp <- GDP$CHN
to <- TO$CHN
up <- UP$CHN
years <- CO2$Years
China <- cbind.data.frame(co2, gdp, to, up, years)
China_ts = ts(China[-5],start = min(China$years),end=max(China$years))

In [ ]:
cat("NAs in CO2:",sum(is.na(China$co2)), 
     "\nNAs in GDP:",sum(is.na(China$gdp)),
     "\nNAs in TO:", sum(is.na(China$to)),
     "\nNAs in UP:", sum(is.na(China$up)),
     "\nNAs in years:", sum(is.na(China$years)))
     

China = China[complete.cases(China),]
row.names(China) <- NULL #reset the index

In [ ]:
options(repr.plot.width=8, repr.plot.height=7)
par(mfrow=c(1,3))
plot(China_ts)

Trends in China begin to grow more or less after 20 years and a particular decrease in the trade feature around 2010 is worth noting.

In [ ]:
acf(China[-5])

Even in the China autocorrelograms, initially, the values of the time series are strongly correlated and then lose significance in the last lags.

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)
norm.Chi <- cbind.data.frame(scale(China[-5]), China$years)

ggplot(norm.Chi, aes(x=China$years))+
        geom_line(aes(y=co2, color="CO2"),na.rm = TRUE) +
        geom_line(aes(y=gdp, color="GDP"),na.rm = TRUE)+
        geom_line(aes(y=to, color="Trade Op"),na.rm = TRUE)+
        geom_line(aes(y=up, color="Urban pop"),na.rm = TRUE)+
        scale_color_discrete(name = "Features", labels = c(
            "#CD5C5C"="CO2", "#4B0082"="GDP", "#000080"="Trade Op", "#7FFF00"="Urban pop"))+
        ylab("Features") +
        ggtitle("Normalized Features")

In [ ]:
options(repr.plot.width=6, repr.plot.height=7)
corr <- cor(China[-5], use="pairwise.complete.obs")

col <- colorRampPalette(c("#BB4444", "#EE9988", "#FFFFFF", "#77AADD", "#4477AA"))
corrplot(corr, method="color", col=col(200), 
         type="lower", order="hclust", 
         addCoef.col = "black", 
         tl.col="black", tl.srt=40, 
         sig.level = 0.01, insig = "blank", 
         diag=FALSE)

The correlation plot is as blue as the sky, showing high multicollinearity between all the variables.

# Modelling 

Models we will implement:

- **Linear Regression Model**, we did not focussed much on explaining these models.

- **Diffusion models**, for instance Bass Model and Generalized Bass Model, we made a detailed explanation of these models and how them works to try understand some possible reasons behind trend's behaviours.

- **Exponential Smoothing and ARIMA**, in particular for ARIMAs, we explored (adding a little of theoretical reasons) how it works and how to differentiate the time series to get a "white noise".

- **Generalized Additive Models (GAM)**, in this case we tried explain (this time without theorethical notions) how differentiating  the explanatory variables help in fitting the response variable.

- **Tree-based methods: Bagging, Boosting**, those models wanted to be a trial and as we can see they will result not suitable for our task.

For some models we are also going to check the autocorrelation by performing test (i.e. Durbin Watson test) and checking the ACF

## Splitting data

For the multivariate modelling, we split the data from the time series start's untill 2015 to fit the models, and the data after 2015 will be forecasted and compared to the original ones.

Meanwhile, for the univariate analysis we are going to use the co2 timeseries we have seen during the EDA phase.

In [ ]:
set.seed(1)
#samp = sample (1:nrow(Italy), 0.7*nrow(Italy))
train=Italy %>% filter(Italy$years >= min(Italy$years) & Italy$years<=2015)
test=Italy %>% filter(Italy$years >= 2015 )

train_usa = USA %>% filter(USA$years >= min(USA$years) & USA$years<=2015)
test_usa = USA %>% filter(USA$years >= 2015 )

train_chn = China %>% filter(China$years >= min(China$years) & China$years<=2015)
test_chn = China %>% filter(China$years >= 2015 )

## Simple Linear Regression

Linear Regression predicts the future values from the past values. It finds a linear relationship between forecast variable CO2 and predictor variables (GDP, UP, and TO). 

### Italy

In [ ]:
MLR <- tslm(formula = co2 ~ gdp + up +to, data=window(Italy_ts, start=1970, end=2016 -.1))
summary(MLR)

Time series linear regression and the multiple linear regression below are exactly the same

In [ ]:
LM <- lm(formula= co2 ~ gdp + up + to, data=train)
summary(LM)
fit <- fitted.values(LM)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)
it.fit <- cbind.data.frame(train$co2,fit, train$years)
colnames(it.fit) <- c("co2", "fitted","years")
ggplot(it.fit, aes(x=years))+
    geom_line(aes(y=co2, color="Real"),na.rm = TRUE) +
    geom_line(aes(y=fitted, color="Fitted"),na.rm = TRUE)

In [ ]:
it.pred <- predict(LM, test)
it.pred <- cbind.data.frame(test$co2, it.pred, test$years)
colnames(it.pred) <- c("co2", "pred", "years")

In [ ]:
## I'm going to plot train and test with fits and predictions on an unique plot, it'll be a little messing
## since the "predict" function doesn't have its autoplot method

ggplot() + 
    geom_line(data=it.fit, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data=it.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
    geom_line(data = it.pred, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data = it.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

In [ ]:
## don't know if we are interested in that too, delete in case
par(mfrow = c(1,4))

plot(LM)

In [ ]:
#analysis of residuals
par(mfrow = c(1,2))
res<- residuals(LM) 
plot(res, type = "b") 
#the form of residuals indicates the presence of positive autocorrelation
Acf(res)
dw<- dwtest(LM, alt="two.sided")
dw

### USA

In [ ]:
LM_usa <- lm(formula= co2 ~ gdp+ up + to, data=train_usa)
summary(LM_usa)
fit_usa <- fitted.values(LM_usa)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)
usa.fit <- cbind.data.frame(train_usa$co2,fit_usa, train_usa$years)
colnames(usa.fit) <- c("co2", "fitted","years")

ggplot(usa.fit, aes(x=years))+
    geom_line(aes(y=co2, color="Real"),na.rm = TRUE) +
    geom_line(aes(y=fitted, color="Fitted"),na.rm = TRUE)

In [ ]:
usa.pred <- predict(LM_usa, test_usa)
usa.pred <- cbind.data.frame(test_usa$co2, usa.pred, test_usa$years)
colnames(usa.pred) <- c("co2", "pred", "years")

ggplot() + 
    geom_line(data=usa.fit, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data=usa.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
    geom_line(data = usa.pred, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data = usa.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

In [ ]:
par(mfrow = c(1,4))

plot(LM_usa)

In [ ]:
#analysis of residuals
par(mfrow = c(1,2))
res_usa<- residuals(LM_usa) 
plot(res_usa, type = "b") 
#the form of residuals indicates the presence of positive autocorrelation
Acf(res_usa)
#and also the dw test
dw_usa<- dwtest(LM_usa, alt="two.sided")
dw_usa

### China

In [ ]:
#MLR_chn <- tslm(formula = co2_chn ~ gdp_chn + up_chn +to_chn, data=window(chn_ts, start=1960, end=2015 -.1))

In [ ]:
LM_chn <- lm(formula= co2 ~ gdp + up + to, data=train_chn)
summary(LM_chn)
fit_chn <- fitted.values(LM_chn)

In [ ]:
options(repr.plot.width=25, repr.plot.height=6)
chn.fit <- cbind.data.frame(train_chn$co2,fit_chn, train_chn$years)
colnames(chn.fit) <- c("co2", "fitted","years")

ggplot(chn.fit, aes(x=years))+
    geom_line(aes(y=co2, color="Real"),na.rm = TRUE) +
    geom_line(aes(y=fitted, color="Fitted"),na.rm = TRUE)

In [ ]:
chn.pred <- predict(LM_chn, test_chn)
chn.pred <- cbind.data.frame(test_chn$co2, chn.pred, test_chn$years)
colnames(chn.pred) <- c("co2", "pred", "years")

ggplot() + 
    geom_line(data=chn.fit, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data=chn.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
    geom_line(data = chn.pred, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data = chn.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

In [ ]:
par(mfrow = c(1,4))

plot(LM_chn)

In [ ]:
#analysis of residuals
par(mfrow = c(1,2))
res_chn<- residuals(LM_chn) 
plot(res_chn, type = "b") 
#the form of residuals indicates the presence of positive autocorrelation
Acf(res_chn)

dw_chn<- dwtest(LM_chn, alt="two.sided")
dw_chn

## Non linear regression model: Bass Model and Generalized Bass Model

The Generalized Bass Model predicts the volume of a new model by using the historical data of other products. It calculates the similarity of the new product and old product and then estimates how new products get adopted in a population.  

As we have seen in the EDA section, CO2 emissions are strictly related to the production of goods and services within the borders of the countries. That in turn is related to economic behaviors, as the high correlation between CO2 and GDP let us understand. So we decided to make use of diffusion models such as BM and GBM, to try to predict the behavior of CO2 trends, even knowing that there are 2 main problems in our series:
- We didn't take the series from the beginning, since there are no data or they are hard to find about CO2 emission before1960, but we limited our series to begin from a storical minimum.
- We already have a lot of data and, in some cases (for developed countries), it seems that the trend already reached a peak in the emissions, that explains the reason behind the good estimations of some models.

In the first point we developed a standard Bass Model for all the countries to get the parameters that we then used to train the GBM, and for this last one we will see more in particular how it fits the data and estimate the parameters. Moreover for different countries we are going to see the shocks we applied to the models and give some possible historical reasoning for the presence of them in the time series (that we didn’t demonstrate with statistics to avoid fall in too much complexity) 


In [ ]:
install.packages("DIMORA")
library(DIMORA)
library(readxl)

### Italy

In [ ]:
##### Bass standard model on CO2 emission to get the coefficents
BM<- BASS.standard(Italy$co2, display=T)
summary(BM)
coef(BM)
BM.fitted <- fitted(BM)
predictedBM <- predict(BM,newx = 45:48)
plot(BM, mode ="i", oos="y", xlim=c(1,50))
lines(train)

check plot below

In [ ]:
#####GBM, 1 exponential shock 
GBMe1<- BASS.generalized(Italy$co2, shock = "mixed", nshock = 2, 
                         prelimestimates = c(40587275.90, 0.0078, 0.024, 39, .05, -.1, 12, 20, -.2),
                         display=F, max.iter=60) # , 45, 0.1, 0.2
summary(GBMe1)
GBM.fitted<- fitted(GBMe1)
coef(GBMe1)
#residuals(GBMe1)
predictedGBM<- predict(GBMe1,newx = 45:48)
plot(GBMe1, mode ="i", oos="y", xlim=c(1,50))

Let’s start with Italy where we modeled 2 shocks:
a negative rectangular between lags 11 and 32
a negative exponential after lag 39, with a slow memory that will take the model to fastly decrease as we will see in a few.

In [ ]:
####graphical inspection of BM e GBMe1
BMS.fitted <- cbind.data.frame(GBM.fitted, BM.fitted, Italy$years)
colnames(BMS.fitted) <- c("gbm", "bm", "years")
# BMS.pred <- cbind.data.frame(predictedGBM, predictedBM, test$years)
# colnames(BMS.pred) <- c("gbm", "bm", "years")

ggplot()+
    geom_line(data=train, aes(x= years,y=co2, color="Actual Train"))+
    geom_line(data=test, aes(x= years,y=co2, color="Actual Test"))+

    geom_line(data=BMS.fitted, aes(x=years,y=gbm, color="GBM"))+
    #geom_line(data=BMS.pred, aes(x=years,y=gbm, color="GBM"))+

    geom_line(data=BMS.fitted, aes(x=years,y=bm, color="BM"))
    #geom_line(data=BMS.pred, aes(x=years,y=bm, color="BM"))

The negative rectangular one goes approximately from 1980 to 2005 where we find a decrease in oil fuel usage together with an increase in gas, a decrease of almost 8% in the industrial sector, and the contribution of nuclear energy that passed from 5% to 15% between 1980 and 1994.  
Then we find a negative exponential shock between 2007 and 2009 that can be addressed, between other factors, to the great financial crisis that interested USA’s markets in that period, followed from an increase in price for petroleum. 
In this period we can also notice a decrease in trade openness (probably affected by the same reasons). 

Before explain the plot: we modeled just two shocks because the DIMORA library has a constraint of just 2 mixed shocks, because there are probably other significant shocks in the series, for example after 2008 there are these 2 “sobs” that the model is not capable to capture after the exponential shock, which makes the fit falling down maybe too prematurely.
On the other hand, the standard BM gives us a longer-term perspective for co2 emissions that seems enough realistic 



In [ ]:
####residual analysis of GBM
resGBM<- residuals(GBMe1)
plot(resGBM, type="b")
Acf(resGBM)
Pacf(resGBM)

The residuals of GBM show us some significant autocorrelation, with 2 opposite peaks in the first two lags of the partial autocorrelation plot.

### USA

In [ ]:
##### Bass standard model on CO2 emission to get the coefficents
BM_usa<- BASS.standard(USA$co2, display=T)
summary(BM_usa)
coef(BM_usa)
BM_usa.fitted <- fitted(BM_usa)
predictedBM_usa <- predict(BM_usa,newx = 45:48)
plot(BM_usa, mode ="i", oos="y", xlim=c(1,50))
lines(train_usa)

In [ ]:
#####GBM, 1 exponential shock 
GBMe1_usa<- BASS.generalized(USA$co2, shock = "mixed", nshock = 2, 
                             prelimestimates = c(680245661, 0.006, 0.017, 39, .3, -.1, 12, 20, -.2),
                             display=F, max.iter=60) # , 45, 0.1, 0.2
summary(GBMe1_usa)
GBM_usa.fitted<- fitted(GBMe1_usa)
coef(GBMe1_usa)
#residuals(GBMe1)
predictedGBM_usa<- predict(GBMe1_usa,newx = 45:48)
plot(GBMe1_usa, mode ="i", oos="y", xlim=c(1,50))

For co2 emissions in the USA we get a similar situation as Italy, and we modeled 2 shocks:
- a negative rectangular one between lag 10 and 25 with an intensity of almost -0.1
- a negative exponential around lag 38 characterized by a memory that, just as Italy, bring the model to a faster decrease, but smoothly compared to Italy as we will see


In [ ]:
####graphical inspection of BM e GBMe1
BMS_usa.fitted <- cbind.data.frame(GBM_usa.fitted, BM_usa.fitted, USA$years)
colnames(BMS_usa.fitted) <- c("gbm", "bm", "years")

ggplot()+
    geom_line(data=train_usa, aes(x= years,y=co2, color="Actual Train"))+
    geom_line(data=test_usa, aes(x= years,y=co2, color="Actual Test"))+

    geom_line(data=BMS_usa.fitted, aes(x=years,y=gbm, color="GBM"))+

    geom_line(data=BMS_usa.fitted, aes(x=years,y=bm, color="BM"))

The first negative rectangular shock verifies between 1979 and 1996, when we find two major events such as the stagflation in 1980 and the increase in oil price from 1974.

The negative exponential shock falls around 2007, where we find again the financial crisis, followed by a huge change in USA’s energy sources characterized by a great decrease in coal usage replaced by natural gas and renewable energy.

By the way in USA’s GBM after the negative exp shock we have a smoother decrease, that seems to be more realistic, meanwhile BM seems to predict a longer horizon for co2 emissions, that hopefully will not happen.


In [ ]:
####residual analysis of GBM
resGBM_usa<- residuals(GBMe1_usa)
plot(resGBM_usa, type="b")
Acf(resGBM_usa)
Pacf(resGBM_usa)

The residuals show us significant autocorrelation, characterized by partial autocorrelation in the first 2 lags

### China

In [ ]:
##### Bass standard model on CO2 emission to get the coefficents
BM_chn<- BASS.standard(China$co2, display=T)
summary(BM_chn)
coef(BM_chn)
BM_chn.fitted <- fitted(BM_chn)
predictedBM_chn <- predict(BM_chn,newx = 50:65)
plot(BM_chn, mode ="i", oos="y", xlim=c(1,65))

In [ ]:
length(China$co2)

In [ ]:
#####GBM, 1 exponential shock 
GBMe1_chn<- BASS.generalized(China$co2, shock = "exp", nshock = 1, 
                         prelimestimates = c(10055728765, 0.001, 0.05, 35, .07,.6),
                         display=F, max.iter=60)
summary(GBMe1_chn)
GBM_chn.fitted<- fitted(GBMe1_chn)
coef(GBMe1_chn)
#residuals(GBMe1)
predictedGBM_chn<- predict(GBMe1_chn,newx = 59:63)
plot(GBMe1_chn, mode ="i", oos="y", xlim=c(1,63))

In China we find a very strange case, but with a positive exponential shock it seems we can describe almost well the trend, with a jump at the 44th lag that bring the model to have a fast increase, but as we can see also a fast decrease.

In [ ]:
####graphical inspection of BM e GBMe1
BMS_chn.fitted <- cbind.data.frame(GBM_chn.fitted, BM_chn.fitted, China$years)
colnames(BMS_chn.fitted) <- c("gbm", "bm", "years")
# BMS_chn.pred <- cbind.data.frame(predictedGBM_chn, predictedBM_chn, test_chn$years)
# colnames(BMS_chn.pred) <- c("gbm", "bm", "years")

ggplot()+
    geom_line(data=train_chn, aes(x= years,y=co2, color="Actual Train"))+
    geom_line(data=test_chn, aes(x= years,y=co2, color="Actual Test"))+

    geom_line(data=BMS_chn.fitted, aes(x=years,y=gbm, color="GBM"))+

    geom_line(data=BMS_chn.fitted, aes(x=years,y=bm, color="BM"))

Here we can see the estimations, and we can see that BM just exploded exponentially, meanwhile GBM after the exponential shock get this bell shaped trend that seems to be very optimistic in co2 decrease.

The same kind of increase we have seen is present also in the GDP and urban population trend in the same years, in which we find that, even if trade openness slows down, it may regard just the inport, because the exports and the production of goods keep increasing hugely from 2003.

In [ ]:
####residual analysis of GBM
resGBM_chn<- residuals(GBMe1_chn)
plot(resGBM_chn, type="b")
Acf(resGBM_chn)
Pacf(resGBM_chn)

The residuals show us an heaving behavior in autocorrelation, that propagates just from 1 single lag.

## Time Series Analysis: ARIMA Model and Exponential Smoothing

### Simple Exponential Smoothing and Holt's Exponential Smoothing

In [ ]:
#simple exponential smoothing 
ses.train <- ses(train$co2,alpha = .2, h = 6)
autoplot(ses.train)

In [ ]:
#holt
holt_train = holt(train$co2, h=6)
autoplot(holt_train)

### ARIMA

In a first place we chose to make use of the auto.arima function to try to find the best model, by the way it was hard to go explain the reasoning of some behaviors. 

So we decided to train them on the co2 series, by manually choosing the order based on the analysis of the residuals autocorrelation for each country, in general that was the rules we applied to find the optimal model:
1. Apply differencing until the residuals become approximately stationary with no long-term trend, that is a tendency to return to its mean
2. Add iteratively 1 unit of AutoRegression or MovingAverage depending on the presence of positive or negative autocorrelation, until obtain white noise


#### Italy

Just for Italy we show the sequence of models we tried

In [ ]:
co2 <- Italy_ts[,1]
plot(co2)
Acf(co2)
Pacf(co2)

The ACF decays linearly, becoming negative and shows a positive autocorrelation until lag 8. PACF shows a significant spike at lag 1 and then nothing alse. This suggest that it may be an ARIMA(p,d,0), In any case, let's explore step to step the differentiations.

In [ ]:
ita.arima1 <- Arima(Italy_ts[,1], order = c(0,1,0)) #trying 1 differentation
ita.resid1 <- residuals(ita.arima1)
tsdisplay(ita.resid1)

This one is the same first step allied for each country, consisting in applying firstly a Random Walk to the series. So, analyzing the autocorrelation, we can observe a rolling behavior in ACF meaning that another differentiation is needed

In [ ]:
ita.arima2 <- Arima(Italy_ts[,1], order = c(0,2,0)) #trying 2nd differentation
ita.resid2 <- residuals(ita.arima2)
tsdisplay(ita.resid2)

with 2 differentiations the ACF presents negative autocorrelation at lag 1, and increasing negative partial AC meaning that we don't need anymore differentiation but we have to increase the moving average (MA)


In [ ]:
ita.arima3 <- Arima(Italy_ts[,1], order = c(0,2,1)) #trying 1 differentation
ita.resid3 <- residuals(ita.arima3)
tsdisplay(ita.resid3)

And boom, we get white noise

In [ ]:
ita.arima3

ita.fitt <- fitted(ita.arima3)
ita.co2 <- zoo(x=Italy_ts[,1], order.by=index(ita.fitt))

for1 <- forecast(ita.arima3)
plot(for1)
lines(ita.fitt, col="red")

#### USA

In [ ]:
co2 <- USA_ts[,1]
plot(co2)
Acf(co2)
Pacf(co2)

In [ ]:
# 1st diff
usa.arima1 <- Arima(USA_ts[,1], order = c(0,1,0))

usa.resid1 <- residuals(usa.arima1)
tsdisplay(usa.resid1)

In [ ]:
# 2st diff, bc we can see in PACF a positive peak
usa.arima2 <- Arima(USA_ts[,1], order = c(0,2,0))

usa.resid2 <- residuals(usa.arima2)
tsdisplay(usa.resid2)

Given the strange behaviour in the autocorrelations and the presence of negative peaks let's apply MA

In [ ]:
# 1st MA
usa.arima2 <- Arima(USA_ts[,1], order = c(0,2,1))
usa.arima2$aic

usa.resid2 <- residuals(usa.arima2)
tsdisplay(usa.resid2)

In [ ]:
usa.arima2 <- Arima(USA_ts[,1], order = c(0,2,2))
usa.arima2$aic

usa.resid2 <- residuals(usa.arima2)
tsdisplay(usa.resid2)

In [ ]:
usa.arima2 <- Arima(USA_ts[,1], order = c(0,2,3))
usa.arima2$aic

usa.resid2 <- residuals(usa.arima2)
tsdisplay(usa.resid2)

In [ ]:
usa.arima3 <- Arima(USA_ts[,1], order = c(0,2,4))
usa.arima3$aic

usa.resid3 <- residuals(usa.arima3)
tsdisplay(usa.resid3)

For USA it seems that the best model is an ARIMA(0,2,3) with an Akaike information criteria score of almost 1521.

In [ ]:
usa.fitt <- fitted(usa.arima2)
usa.co2 <- zoo(x=USA_ts[,1], order.by=index(usa.fitt))

for1 <- forecast(usa.arima2)
plot(for1)
lines(usa.fitt, col="red")

#### China

In [ ]:
co2 <- China_ts[,1]
plot(co2)
Acf(co2)
Pacf(co2)

In [ ]:
#1st diff
chi.arima1 <- Arima(China_ts[,1], order = c(0,1,0))
chi.arima1$aic

chi.resid1 <- residuals(chi.arima1)
tsdisplay(chi.resid1)

In [ ]:
#2st diff
chi.arima1 <- Arima(China_ts[,1], order = c(0,2,0))
chi.arima1$aic

chi.resid1 <- residuals(chi.arima1)
tsdisplay(chi.resid1)

In [ ]:
#negative autocorr -> +1 MA
#1st diff
chi.arima1 <- Arima(China_ts[,1], order = c(0,2,1))
chi.arima1$aic

chi.resid1 <- residuals(chi.arima1)
tsdisplay(chi.resid1)

In [ ]:
#positive autocorr -> +1 AR
#1st diff
chi.arima1 <- Arima(China_ts[,1], order = c(1,2,1))
chi.arima1$aic

chi.resid1 <- residuals(chi.arima1)
tsdisplay(chi.resid1)

In [ ]:
#1st diff
chi.arima1 <- Arima(China_ts[,1], order = c(3,4,3))
chi.arima1$aic

chi.resid1 <- residuals(chi.arima1)
tsdisplay(chi.resid1)

#chi.arima1 <- Arima(China_ts[,1], order = c(2,2,1)) AIC 1538.0096893258
#chi.arima1 <- Arima(China_ts[,1], order = c(3,3,2)) 1522.77906064584
#chi.arima1 <- Arima(China_ts[,1], order = c(2,3,2)) 1521.28108049291
#chi.arima1 <- Arima(China_ts[,1], order = c(3,3,3)) 1520.74760160395
#chi.arima1 <- Arima(China_ts[,1], order = c(2,3,1)) AIC 1519.91858019209

Estimating China’s ARIMA model was more challenging because the autocorrelations kept jumping from positive to negative, in fact we had to try something like 10 models, at the end we just decided to stick with an ARIMA(3,4,3) with AIC score of 1514.

In [ ]:
chi.arima1
chi.fitt <- fitted(chi.arima1)
chi.co2 <- zoo(x=China_ts[,1], order.by=index(chi.fitt))

for1 <- forecast(chi.arima1)
plot(for1)
lines(chi.fitt, col="red")

## Generalized Additive Models (GAM)
For the GAM, we chose the models manually by looking at the significance of both the parametric, nonparametric effect on the variables and the Akaike Information Criteria
In our GAMs we used both splines and loess, leaving the default parameters. In the same order as before, we are going to present the GAMs that better fit our data and the AIC score of every model we tried.


In [ ]:
library(gam)

### Italy

simpler gam without smoothing splines or loess

In [ ]:
#Values for df should be greater than 1, with df=1 implying a linear fit
gi0 <- gam(co2~gdp + up + to, data=train)
summary(gi0)

In [ ]:
par(mfrow=c(2,2))
plot(gi0, se=T)
AIC(gi0)
####GAM best model

2nd GAM with smoothing splines aplied to all the variables, we can notice an high increase in the accuracy of the fitting due to the non linearity of the explanatory variables, by the way it seems that the smoothing spline on *Urban Population* increased it's significativity, on the contrary the one of *Trade Openess* decreases a lot in the *Non Parametric Effect*

In [ ]:
gi1 <- gam(co2~ s(gdp) + s(up) + s(to), data=train)
summary(gi1)

In [ ]:
par(mfrow=c(2,2))
plot(gi1, se=T)
AIC(gi1)

applying loess the AIC decrease and the parametric effect of *Urban Population* increased in significance, in the next model I will try merge g1 and g2, to try get the best model

In [ ]:
gi2 <- gam(co2~ lo(gdp) + lo(up) + lo(to), data=train)
summary(gi2)

In [ ]:
par(mfrow=c(2,2))
plot(gi2, se=T)
AIC(gi2)

in this last model (g3) we achieve the best AIC (i.e. 867.3228) and due to the multicollinearity between the explanatory variables (maybe) the significativity of all the three has reach an high level compared to the other models 

In [ ]:
gi3 <- gam(co2~ s(gdp) + lo(up) + to, data=train)
summary(gi3)

In [ ]:
par(mfrow=c(2,2))
plot(gi3, se=T)
AIC(gi3)

it seems that the between the residuals there is almost no autocorrelation (**apply DW test and calculate VIF**) meaning that we have a *white noise*

In [ ]:
tsdisplay(residuals(gi3))
dwtest(gi3)

Now we are going to try fit the data with this last model firstly in the classical way, then applying ARIMA (auto.arima on the model's residuals) to try adjust the fitting (as we saw in *Lab_GAM*)

In [ ]:
gi3.fit <- fitted(gi3)
padding.len <- nrow(train)-length(gi3.fit)

plot(train$co2)
lines(gi3.fit, col=2)

Actually the GAM do a perfect fitting of the data, but it is for sure falling in overfitting, let's predict on the test set and see what happen

In [ ]:
gi3.fit <- cbind.data.frame(train$co2,gi3.fit, train$years)
colnames(gi3.fit) <- c("co2", "fitted","years")

gi3.pred <- predict(gi3, test)
gi3.pred <- cbind.data.frame(test$co2, gi3.pred, test$years)
colnames(gi3.pred) <- c("co2", "pred", "years")

In [ ]:
ggplot() + 
    geom_line(data=gi3.fit, aes(x=years, y=co2, color='Actual Train'), na.rm=TRUE) +
    geom_line(data=gi3.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
    geom_line(data = gi3.pred, aes(x=years, y=co2, color='Actual Test'), na.rm=TRUE) +
    geom_line(data = gi3.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

As we suspected the GAM model overfits a lot, resulting in a misclassification on the test set.

let us now try the auto.arima on the model, just to see if it can do even a little better

In [ ]:
aar.gi3 <- auto.arima(residuals(gi3))
summary(aar.gi3)

ARIMA(1,0,0) = first-order autoregressive model: if the series is stationary and autocorrelated, perhaps it can be predicted as a multiple of its own previous value, plus a constant.  The forecasting equation in this case is

Ŷt  =  μ  +  ϕ1Yt-1

…which is Y regressed on itself lagged by one period. This is an “ARIMA(1,0,0)+constant” model.  **If the mean of Y is zero, then the constant term would not be included.**

**If the slope coefficient ϕ1 is positive and less than 1 in magnitude (it must be less than 1 in magnitude if Y is stationary), the model describes mean-reverting behavior in which next period’s value should be predicted to be ϕ1 times as far away from the mean as this period’s value.**  If ϕ1  is negative, it predicts mean-reverting behavior with alternation of signs, i.e., it also predicts that Y will be below the mean next period if it is above the mean this period.

In a second-order autoregressive model (ARIMA(2,0,0)), there would be a Yt-2  term on the right as well, and so on.  Depending on the signs and magnitudes of the coefficients, an ARIMA(2,0,0) model could describe a system whose mean reversion takes place in a sinusoidally oscillating fashion, like the motion of a mass on a spring that is subjected to random shocks.

In [ ]:
# this library collide with the gam one
# library(mgcv)

Then we apply the result obtained before to the GAM

In [ ]:
# gi4 <- gamm(co2~ s(gdp) + lo(up) + to,
#             correlation=corARMA(p = 1), data=train)
# gi4

In [ ]:
# res <- resid(gi4$lme, type = "normalized")
# acf(res, lag.max = 36, main = "ACF - AR(1) errors")
# pacf(res, lag.max = 36, main = "pACF- AR(1) errors")

In [ ]:
# gi4.fit <- fitted(gi4$gam)
# gi4.fit <- cbind.data.frame(train$co2,gi4.fit, train$years)
# colnames(gi4.fit) <- c("co2", "fitted","years")

# gi4.pred <- predict(gi4$gam, test)
# gi4.pred <- cbind.data.frame(test$co2, gi4.pred, test$years)
# colnames(gi4.pred) <- c("co2", "pred", "years")

In [ ]:
# ggplot() + 
#     geom_line(data=gi4.fit, aes(x=years, y=co2, color='Actual Train'), na.rm=TRUE) +
#     geom_line(data=gi4.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
#     geom_line(data = gi4.pred, aes(x=years, y=co2, color='Actual Test'), na.rm=TRUE) +
#     geom_line(data = gi4.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

It seems that ARMA get the situation even worse

### USA

In [ ]:
#Values for df should be greater than 1, with df=1 implying a linear fit
gi0_usa <- gam(co2 ~ gdp + up + to, data=train_usa)
summary(gi0_usa)

In [ ]:
par(mfrow=c(2,2))
plot(gi0_usa, se=T)
AIC(gi0_usa)

In [ ]:
gi1_usa <- gam(co2~ s(gdp) + s(up) + s(to), data=train_usa)
summary(gi1_usa)

In [ ]:
par(mfrow=c(2,2))
plot(gi1_usa, se=T)
AIC(gi1_usa)

In [ ]:
gi2_usa <- gam(co2~ lo(gdp) + lo(up) + lo(to), data=train_usa)
summary(gi2_usa)

In [ ]:
par(mfrow=c(2,2))
plot(gi2_usa, all.terms = TRUE, se=T)
AIC(gi2_usa)

In [ ]:
gi3_usa <- gam(co2~ lo(gdp) + s(up) + s(to), data=train_usa)
summary(gi3_usa)

In [ ]:
par(mfrow=c(2,2))
plot(gi3_usa, se=T)
AIC(gi3_usa)

In [ ]:
tsdisplay(residuals(gi3_usa))
dwtest(gi3_usa)

In [ ]:
gi3_usa.fit <- fitted(gi3_usa)
padding.len_usa <- nrow(train_usa)-length(gi3_usa.fit)
# NAs <- rep(NA, padding.len)
# g3.fit <- c(NAs, g3.fit)

plot(train_usa$co2)
lines(gi3_usa.fit, col=2)

In [ ]:
gi3_usa.fit <- cbind.data.frame(train_usa$co2,gi3_usa.fit, train_usa$years)
colnames(gi3_usa.fit) <- c("co2", "fitted","years")

gi3_usa.pred <- predict(gi3_usa, test_usa)
gi3_usa.pred <- cbind.data.frame(test_usa$co2, gi3_usa.pred, test_usa$years)
colnames(gi3_usa.pred) <- c("co2", "pred", "years")

In [ ]:
ggplot() + 
    geom_line(data=gi3_usa.fit, aes(x=years, y=co2, color='Actual Train'), na.rm=TRUE) +
    geom_line(data=gi3_usa.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
    geom_line(data = gi3_usa.pred, aes(x=years, y=co2, color='Actual Test'), na.rm=TRUE) +
    geom_line(data = gi3_usa.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

In [ ]:
aar.gi3_usa <- auto.arima(residuals(gi3_usa))
summary(aar.gi3_usa)

In [ ]:
# gi4_usa <- gamm(co2~ lo(gdp) + s(up) + s(to),
#             correlation=corARMA(p = 1), data=train_usa)
# gi4_usa

# res <- resid(gi4_usa$lme, type = "normalized")
# acf(res, lag.max = 36, main = "ACF - AR(1) errors")
# pacf(res, lag.max = 36, main = "pACF- AR(1) errors")
# gi4_usa.fit <- fitted(gi4_usa$gam)

# gi4_usa.fit <- cbind.data.frame(train_usa$co2,gi4_usa.fit, train_usa$years)
# colnames(gi4.fit) <- c("co2", "fitted","years")

# gi4_usa.pred <- predict(gi4_usa$gam, test_usa)
# gi4_usa.pred <- cbind.data.frame(test_usa$co2, gi4_usa.pred, test_usa$years)
# colnames(gi4_usa.pred) <- c("co2", "pred", "years")

# ggplot() + 
#     geom_line(data=gi4_usa.fit, aes(x=years, y=co2, color='Actual Train'), na.rm=TRUE) +
#     geom_line(data=gi4_usa.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
#     geom_line(data = gi4_usa.pred, aes(x=years, y=co2, color='Actual Test'), na.rm=TRUE) +
#     geom_line(data = gi4_usa.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE)

Confirmed with the DW test, the residuals were significantly correlated. Just as what we did in Italy’s GAM, we tried to apply a Moving Average Unit to the correlation, but we could not get a good result

### China

In [ ]:
#Values for df should be greater than 1, with df=1 implying a linear fit
gi0_chn <- gam(co2~gdp + up + to, data=train_chn)
summary(gi0_chn)

In [ ]:
par(mfrow=c(2,2))
plot(gi0_chn, se=T)
AIC(gi0_chn)
####GAM best model

In [ ]:
gi1_chn <- gam(co2~ s(gdp) + s(up) + s(to), data=train_chn)
summary(gi1_chn)

In [ ]:
par(mfrow=c(2,2))
plot(gi1_chn, se=T)
AIC(gi1_chn)

In [ ]:
gi2_chn <- gam(co2~ lo(gdp) + lo(up) + lo(to), data=train_chn)
summary(gi2_chn)

In [ ]:
par(mfrow=c(2,2))
plot(gi2_chn, se=T)
AIC(gi2_chn)

In [ ]:
gi3_chn <- gam(co2~ s(gdp) + lo(up) + s(to), data=train_chn)
summary(gi3_chn)

In [ ]:
par(mfrow=c(2,2))
plot(gi3_chn, se=T)
AIC(gi3_chn)

In [ ]:
tsdisplay(residuals(gi3_chn))
dwtest(gi3_chn)

The residuals again seem to be positively autocorrelated

In [ ]:
gi3_chn.fit <- fitted(gi3_chn)
padding_chn.len <- nrow(train_chn)-length(gi3_chn.fit)
# NAs <- rep(NA, padding.len)
# g3.fit <- c(NAs, g3.fit)

plot(train_chn$co2)
lines(gi3_chn.fit, col=2)

In [ ]:
gi3_chn.fit <- cbind.data.frame(train_chn$co2,gi3_chn.fit, train_chn$years)
colnames(gi3_chn.fit) <- c("co2", "fitted","years")

gi3_chn.pred <- predict(gi3_chn, test_chn)
gi3_chn.pred <- cbind.data.frame(test_chn$co2, gi3_chn.pred, test_chn$years)
colnames(gi3_chn.pred) <- c("co2", "pred", "years")

In [ ]:
ggplot() + 
    geom_line(data=gi3_chn.fit, aes(x=years, y=co2, color='Actual Train'), na.rm=TRUE) +
    geom_line(data=gi3_chn.fit, aes(x=years, y=fitted, color='Fitted'), na.rm=TRUE) +
    geom_line(data = gi3_chn.pred, aes(x=years, y=co2, color='Actual Test'), na.rm=TRUE) +
    geom_line(data = gi3_chn.pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) ## TODO: should be added a confidence intervall

In [ ]:
aar.gi3_chn <- auto.arima(residuals(gi3))
summary(aar.gi3_chn)

### Summing up:
* For Italy we applied cubic spline on GDP and loess on urban population with polynomial degree = 4.6, leaving trade openness linear. We could see that all the linear relationships were significant for the model together with the non-parametric transformations applied to GDP and urban population.

* For the USA, the best model comprehends loess on GDP with almost quadratic degrees, and cubic splines applied to urban population and trade openness. The urban population did not show significance to the parametric effect, because of the multicollinearity with the other variables, but results to be very significant for the non parametric effect. But at the same time the trade openness decreased. To better fit the USA’s GAM, we probably had to apply different parameters on the different regression methods.

* For China, we used cubic spline on gdp, quadratic loess on urban population and linear trade openness. It seem to be the best approach in this multivariate regression, with both parametric and non parametric effects significant


## Bagging, Boosting

### Bagging

The bagging model take b bootstrapped samples from the training dataset, and build a decision tree for each bootstrapped sample,then average the predictions of each tree to come up with a final model. 

In [ ]:
library(e1071)       #for calculating variable importance
library(rpart)       #for fitting decision trees
library(ipred)       #for fitting bagged decision trees

- nbagg = number of bootstrapped samples to build the bagged model
- coob to be TRUE to obtain the estimated out-of-bag error
- minsplit = 2 This tells the model to only require 2 observations in a node to split.
- cp = 0 This is the complexity parameter. By setting it to 0, we don’t require the model to be able to improve the overall fit by any amount in order to perform a split.

In [ ]:
#make this example reproducible
set.seed(1)

#fit the bagged model
bag <- bagging(
  formula = co2 ~ gdp + to + up,
  data = train,
  nbagg = 150,   
  coob = TRUE,
  control = rpart.control(minsplit = 2, cp = 0)
)
bag

In [ ]:
#use fitted bagged model to predict test dataset
predict = predict(bag, newdata=test)
predict
pred <- cbind.data.frame(test$co2, predict, test$years)
colnames(pred) <- c("co2", "pred", "years")

In [ ]:
ggplot() + 
    geom_line(data = train, aes(x=years, y=co2, color='Actual Train'), na.rm=TRUE) +
    geom_line(data = pred, aes(x=years, y=co2, color='Actual Test'), na.rm=TRUE) +
    geom_line(data = pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE) 

### Boosting

Gradient Boosting produces a prediction model in the form of the ensemble of weak prediction models, for instance, decision trees. However, combining the weak predictors together gives a strong model.

needa check what wrong with the model  
**try other loss functions**

In [ ]:
library(gbm)

In [ ]:
#fit the boost model
boost <- gbm(
  formula = co2 ~ gdp + to + up + years,
  data = train,
  n.trees = 100)
summary(boost)

In [ ]:
boost

seems that *years* has 0 influence on the model, let's fit a model without the *years* variable:

In [ ]:
boost <- gbm(
  formula = co2 ~ gdp + to + up,
  data = train,
  n.trees = 100)
summary(boost)
boost

In [ ]:
gboost.pred <- predict(boost, newdata = test)
gboost.pred
pred <- cbind.data.frame(test$co2, gboost.pred, test$years)
colnames(pred) <- c("co2", "pred", "years")

In [ ]:
ggplot() + 
    geom_line(data = train, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data = pred, aes(x=years, y=co2, color='Actual'), na.rm=TRUE) +
    geom_line(data = pred, aes(x=years, y=pred, color='Predicted'), na.rm=TRUE)

As we can see, a constant value was predicted over the whole testing period. We thought that it was not a good forecasting method, so we did not further consider using this model. The same goes for the Bagging method.


# Conclusion

In conclusion, Non Linear Diffusion Models, such as BM and GBM gave us good estimation of the future possible evolution in co2 emissions.  
GAM was the most suitable models to make multivariate time series analysisARIMA models results to work nicely in predicting the trend of the co2 time series.  
